In [1]:
from src.Model import Model
import torch
import torch.nn.functional as F
from tqdm import tqdm
from src.Tools import generate_text,get_eos_embedding,update_sequence_mask,get_one_hot
from src.Train import one_epoch_caption,one_epoch_QA,train_caption,train_QA
from src.Dataset import QADataset,CaptionDataset
from torch.utils.data import DataLoader
from src.Mean import ImagesDataset

version = 1
model = Model(_model='BERT')

Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['embeddings.mask_token']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
QA_dataset = QADataset('Data/Annotations/QA/train.json',small=True)
Caption_dataset = CaptionDataset('Data/Annotations/Caption/train.json',small=True)
data_loader_cap = DataLoader(Caption_dataset, batch_size= 2, shuffle = True)
data_loader_qa = DataLoader(QA_dataset,batch_size=2,shuffle=True)

Reusing cached mean and std
loading annotations into memory...
Done (t=0.10s)
creating index...
index created! imgs = 23431, anns = 100575


In [3]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#loss_c = train_caption(20,model,data_loader_cap,optimizer,version=version)
loss_q = train_QA(20,model,data_loader_qa,optimizer,version=version)

cuda


  0%|                                 | 0/2 [00:00<?, ?it/s]

generator.embeddings.word_embeddings.weight: 0.0013559464132413268
generator.embeddings.position_embeddings.weight: 0.0018837092211470008
generator.embeddings.token_type_embeddings.weight: 0.012340676970779896
generator.embeddings.LayerNorm.weight: 0.0006705312989652157
generator.embeddings.LayerNorm.bias: 0.000982067664153874
generator.encoder.layer.0.attention.self.query.weight: 0.0006871020887047052
generator.encoder.layer.0.attention.self.query.bias: 3.980559267802164e-05
generator.encoder.layer.0.attention.self.key.weight: 0.0008250973769463599
generator.encoder.layer.0.attention.self.key.bias: 1.7151072229104614e-12
generator.encoder.layer.0.attention.self.value.weight: 0.00529502471908927
generator.encoder.layer.0.attention.self.value.bias: 0.00048423538100905716
generator.encoder.layer.0.attention.output.dense.weight: 0.005018061492592096
generator.encoder.layer.0.attention.output.dense.bias: 0.0006170115084387362
generator.encoder.layer.0.attention.output.LayerNorm.weight: 0.0

 50%|████████████▌            | 1/2 [00:32<00:32, 32.61s/it]


KeyboardInterrupt: 

In [ ]:
for param in model.parameters():
    if param.grad is not None:
        print(param.grad.norm())
    else:
        #print(name, None)
        pass

tensor(6.6333e-22, device='cuda:0')
tensor(7.6920e-21, device='cuda:0')
tensor(2.6131e-21, device='cuda:0')
tensor(3.5901e-21, device='cuda:0')
tensor(3.1773e-21, device='cuda:0')
tensor(3.2438e-21, device='cuda:0')
tensor(9.9041e-23, device='cuda:0')
tensor(3.6790e-20, device='cuda:0')
tensor(0., device='cuda:0')
tensor(6.1624e-22, device='cuda:0')
tensor(0., device='cuda:0')
tensor(2.9312e-20, device='cuda:0')
tensor(0., device='cuda:0')
tensor(4.2745e-21, device='cuda:0')
tensor(9.8899e-22, device='cuda:0')
tensor(2.9144e-20, device='cuda:0')
tensor(3.5905e-22, device='cuda:0')
tensor(1.7250e-19, device='cuda:0')
tensor(2.2770e-22, device='cuda:0')
tensor(3.0343e-19, device='cuda:0')
tensor(2.3462e-20, device='cuda:0')
tensor(2.5228e-19, device='cuda:0')
tensor(1.2804e-20, device='cuda:0')
tensor(1.7864e-18, device='cuda:0')
tensor(0., device='cuda:0')
tensor(3.9656e-20, device='cuda:0')
tensor(1.9372e-21, device='cuda:0')
tensor(4.3382e-19, device='cuda:0')
tensor(1.4469e-21, devic